# Setup

In [ ]:
import pandas as pd
import numpy as np

import pickle
import os

import ipywidgets as widgets
import traitlets
from IPython.display import display, clear_output, HTML

from feature_utils import ItemSelector, TextStats

In [ ]:
data_path = '/home/jovyan/work/data'

In [ ]:
try:
    file_path = os.path.join(data_path, 'helpfulness_estimators.pickle')
    with open(file_path, 'rb') as f:
        estimators = pickle.load(f)
except FileNotFoundError as e:
    print('Please train the estimators first!')
    raise

In [ ]:
reviews = pd.read_csv(os.path.join(data_path, '../data/Reviews.csv'))
reviews['Helpfulness'] = reviews['HelpfulnessNumerator'].divide(reviews['HelpfulnessDenominator'], axis=0)

In [ ]:
# interface controls

widget_summary = widgets.Textarea(
    value='',
    description='Summary',
)

widget_text = widgets.Textarea(
    value='',
    description='Text',
)

widget_score = widgets.Dropdown(
    options=list(range(1,6)),
    value=3,
    description='Score'
)

widget_output = widgets.HTML(
    value=''
)

widget_predict = widgets.Button(description='Predict usefulness')

In [ ]:
def predict_on_click(*args):
    
    # build predictors array
    s = {}
    s['Summary'] = widget_summary.value
    s['Text'] = widget_text.value
    s['Score'] = widget_score.value
    data = pd.DataFrame([s])
    
    results = []
    for name, estimator in estimators.items():
        v = estimator.predict(data)
        results.append('<p>{} -- score: {:.2f}</p>'.format(name, float(v)))
    
    widget_output.value = '\n'.join(results)

widget_predict.on_click(predict_on_click)

In [ ]:
display(widgets.VBox([widget_summary, widget_text, widget_score, widget_predict]))

# Used estimators

The following are the configurations of the best estimators in each class.

In [ ]:
for name, estimator in estimators.items():
    print(name, estimator.steps, sep='\n', end='\n\n\n')

# Examples

You can copy the values from one of these examples.

In [ ]:
examples = reviews.loc[~reviews['Helpfulness'].isnull()].sample(n=5, random_state=0)
for i, s in examples.iterrows():
    print('Helpfulness: {:.2f} ---------------------------------'.format(s['Helpfulness']))
    print('>> Summary', s['Summary'],
          '>> Text', s['Text'],
          '>> Score', s['Score'],
          sep='\n\n', end='\n\n\n\n')

# Results

In [ ]:
display(widget_output)